In [2]:
from pathlib import Path
import os, sys
import django
PROJECTPATH = str(Path.cwd().parent)
sys.path.insert(0, PROJECTPATH)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "mock.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.chdir(PROJECTPATH)
django.setup()

In [3]:
from v1.exam.serializers import  QuestionSolutionSerializer
from v1.exam.models import ExamHistory, ExamSession, ExamType
from rest_framework import serializers
from rest_framework.response import Response
import json
session_id = "ENR000003000004"
session_inst = ExamSession.objects.get(session_id=session_id)


In [10]:
class AnswerResponseSerializer(serializers.ModelSerializer):
    question_details = QuestionSolutionSerializer(source="question")
    selected_answer = serializers.CharField()
    is_correct = serializers.CharField()

    class Meta:
        model = ExamHistory
        fields = (
            "question_details",
            "selected_answer",
            "is_correct",
        )

class ExamSessionSerializer(serializers.ModelSerializer):
    exam_date = serializers.SerializerMethodField()
    other_data = serializers.SerializerMethodField()
    class Meta:
        model = ExamSession
        fields = (
            "session_id",
            "user_exam_type",
            "time_consumed",
            "status",
            "exam_date",
            "other_data"
        )

    def get_exam_date(self, obj):
        return obj.exam_histories.last().exam_date
    
    def get_other_data(self, obj):
        query_set = obj.exam_histories.all()
        return AnswerResponseSerializer(
                query_set,
                many=True,
            ).data

# query_set = ExamHistory.objects.filter(exam_session__session_id=session_id)

# serializer = AnswerResponseSerializer(
#                 query_set,
#                 many=True,
#             )
serializer2 = ExamSessionSerializer(session_inst)

# json.dumps(serializer2.data)

In [11]:
serializer2.data

{'session_id': 'ENR000003000004', 'user_exam_type': 5, 'time_consumed': 3600, 'status': 'C', 'exam_date': datetime.datetime(2023, 4, 20, 13, 49, 16, 909631, tzinfo=datetime.timezone.utc), 'other_data': [OrderedDict([('question_details', OrderedDict([('id', 14), ('question', 'This is a sample question for FUNCTION for 2 difficulty level with no attachment'), ('option_a', 'this is option a'), ('option_b', 'this is option b'), ('option_c', 'this is option c'), ('option_d', 'this is option d'), ('topic_id', 4), ('topic_name', 'FUNCTION'), ('subject_name', 'MATHEMATICS'), ('marks', 1), ('has_attachment', False), ('question_attachments', []), ('solutions', OrderedDict([('id', 14), ('question_id', 14), ('correct_answer', 'c'), ('detailed_answer', 'This is a question 14 solution'), ('has_attachment', True), ('hint_answer', None), ('solution_attachments', [OrderedDict([('name', 'path/for/solution_id14/for/topic/FUNCTION'), ('attachment_url', 'http://mocktest.commedia/path/for/solution_id14/for/

In [19]:
session_inst.exam_histories.last().exam_date

datetime.datetime(2023, 4, 20, 13, 49, 16, 909631, tzinfo=datetime.timezone.utc)

In [ ]:
a = session_inst.exam_histories.all()